In [ ]:
import os
import sys
import matplotlib.pyplot as plt
current_dir = os.getcwd()
parent_parent_dir = os.path.abspath(os.path.join(current_dir, '../..')) # tweak so that you get dir of code project

sys.path.append(parent_parent_dir)

In [ ]:
import json
import numpy as np
import pandas as pd
import ast
import seaborn as sns
%matplotlib inline

In [ ]:
from src.models.optics.get_clusters_from_optics_labels import get_clusters_from_optics_labels
from src.visualization.render_vehicle_track_cluster_in_notebook import render_vehicle_track_cluster_in_notebook

def plot_dtw_distance_matrix_heatmap(intersection_name, params_uid=None):

    print(f'NORMAL {intersection_name}')

    # get data
    data_path = f'{parent_parent_dir}/data/processed/{intersection_name}_cuid.csv'
    df_cuid = pd.read_csv(data_path)
    df_cuid_grouped_path = data_path.replace('.csv', '_grouped.csv')
    df_cuid_grouped = pd.read_csv(df_cuid_grouped_path)
    df_cuid_grouped['x'] = df_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
    df_cuid_grouped['y'] = df_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))


    def get_optics_params_from_metric(optics_params, metric):
        optics_params = optics_params[metric]
        dtw_dist_matrix_key = optics_params['dtw_key']
        max_eps = optics_params['epsilon']
        min_samples = optics_params['min_samples']
        cluster_method = optics_params['cluster_method']
        xi = optics_params['xi']
        kwargs = {
            'max_eps': max_eps,
            'min_samples': min_samples,
            'cluster_method': cluster_method,
            'xi': xi
        }
        filtered_kwargs = {k: v for k, v in kwargs.items() if v is not None}
        return dtw_dist_matrix_key, filtered_kwargs
    
    # check for params uid 
    if params_uid is not None:
        opt_params_path = f'{parent_parent_dir}/data/processed/{intersection_name}_optics_vehicle_paths_optimized_params_{params_uid}.json'
    else:
        opt_params_path = f'{parent_parent_dir}/data/processed/{intersection_name}_optics_vehicle_paths_optimized_params.json'
    with open(opt_params_path, 'r') as f:
        optics_params = json.load(f)

    # define optics params, dtw distance matrix key
    dtw_key_silhouette, kwargs_silhouette = get_optics_params_from_metric(optics_params, 'silhouette')
    dtw_key_davies_bouldin, kwargs_davies_bouldin = get_optics_params_from_metric(optics_params, 'davies_bouldin')
    dtw_key_calinski_harabasz, kwargs_calinski_harabasz = get_optics_params_from_metric(optics_params, 'calinski_harabasz')

    # import dtw distance matrix
    dtw_distance_matrix_path = f'{parent_parent_dir}/data/processed/{intersection_name}_diff_itakura_slope_dtw_matrices.json'
    with open(dtw_distance_matrix_path) as f:
        dtw_distance_matrix = json.load(f)

    # get the distance matrix
    dtw_dist_matrix_silhouette = dtw_distance_matrix[dtw_key_silhouette]
    dtw_dist_matrix_davies_bouldin = dtw_distance_matrix[dtw_key_davies_bouldin]
    dtw_dist_matrix_calinski_harabasz = dtw_distance_matrix[dtw_key_calinski_harabasz]
    print(f'Silhouette Distance Matrix: {dtw_dist_matrix_silhouette}')
    print(f'Davies Bouldin Distance Matrix: {dtw_dist_matrix_davies_bouldin}')
    print(f'Calinski Harabasz Distance Matrix: {dtw_dist_matrix_calinski_harabasz}')
    

    # plot models next to each other
    fig, axs = plt.subplots(1, 3, figsize=(30,10))
    fig.suptitle(f'Intersection: {intersection_name}')

    # Plot the matrix on the provided Axes object using imshow
    dtw_dist_matrix_silhouette = np.array(dtw_dist_matrix_silhouette)
    dtw_dist_matrix_calinski_harabasz = np.array(dtw_dist_matrix_calinski_harabasz)
    dtw_dist_matrix_davies_bouldin = np.array(dtw_dist_matrix_davies_bouldin)
    np.fill_diagonal(dtw_dist_matrix_silhouette, np.nan)
    np.fill_diagonal(dtw_dist_matrix_calinski_harabasz, np.nan)
    np.fill_diagonal(dtw_dist_matrix_davies_bouldin, np.nan)
    cax = axs[0].imshow(dtw_dist_matrix_silhouette, cmap='coolwarm', vmin=0, vmax=500)
    fig.colorbar(cax, ax=axs[0])
    cax = axs[1].imshow(dtw_dist_matrix_davies_bouldin, cmap='coolwarm', vmin=0, vmax=500)
    fig.colorbar(cax, ax=axs[1])
    cax = axs[2].imshow(dtw_dist_matrix_calinski_harabasz, cmap='coolwarm', vmin=0, vmax=500)
    fig.colorbar(cax, ax=axs[2])

    dtw_dist_matrices_keys = [dtw_key_silhouette, dtw_key_davies_bouldin, dtw_key_calinski_harabasz]
    for i, optics_eval_metric_key in enumerate(['silhouette', 'davies_bouldin', 'calinski_harabasz']):
        axs[i].set_title(f'{optics_eval_metric_key}' + '\n' + f'{dtw_dist_matrices_keys[i]}' + '\n' + f'{params_uid if params_uid is not None else ""}')

In [ ]:
# define intersectoin names
intersection_names = ['k729_2022', 'k733_2020', 'k733_2018']

In [ ]:
for intersection_name in intersection_names:
    plot_dtw_distance_matrix_heatmap(intersection_name)

In [ ]:
# choose 5 random dtw distance matrices to plot
intersection_name = 'k729_2022'
dtw_distance_matrix_path = f'{parent_parent_dir}/data/processed/{intersection_name}_diff_itakura_slope_dtw_matrices.json'
with open(dtw_distance_matrix_path) as f:
    dtw_distance_matrix = json.load(f)

dtw_keys = list(dtw_distance_matrix.keys())
dtw_keys = np.random.choice(dtw_keys, 5)
# plot models next to each other
fig, axs = plt.subplots(2, 3, figsize=(30,10))
fig.suptitle(f'Intersection: {intersection_name}')

for i, dtw_key in enumerate(dtw_keys):
    dtw_dist_matrix = np.array(dtw_distance_matrix[dtw_key])
    np.fill_diagonal(dtw_dist_matrix, np.nan)
    cax = axs[i//3, i%3].imshow(dtw_dist_matrix, cmap='coolwarm', vmin=0, vmax=500)
    fig.colorbar(cax, ax=axs[i//3, i%3])
    axs[i//3, i%3].set_title(f'{dtw_key}')